# Project 2 MSI - Ariadna Marín

In [ ]:
from htmd import *
#htmd.config(viewer='ngl')
import os 
from scipy.ndimage.filters import gaussian_filter

In [ ]:
def generate_mat(mol,mol_ref):
    '''Function to generate normalized count matrix from molecule object'''
    mat = np.zeros((myrange,myrange,myrange))
    # center and wrap waters
    mol.wrap('protein') # avoid waters to expand
    mol.align('protein',mol_ref) # align all prots
    # get Oxigen from waters
    watermol = mol.copy()
    watermol.filter('name OH2')
    # modify abnormal points
    s1 = (watermol.coords < myrange/2) # bigger than max cube
    s2 = (watermol.coords > -myrange/2) # smaller than min cube
    m1 = watermol.coords * s1
    mwater = m1 * s2
    # get matrix
    for i in range(0,watermol.coords.shape[0]):
        for t in range(0,watermol.coords.shape[2]):
            x = int(round(mwater[i][0][t]+myrange/2))
            y = int(round(mwater[i][1][t]+myrange/2))
            z = int(round(mwater[i][2][t]+myrange/2))    
            mat[x][y][z] += 1
    mat_norm = (10**-20 + mat) / (watermol.coords.shape[0]*watermol.coords.shape[2])
    return mat_norm

In [ ]:
# constants for G computation
kB = 0.001987191
T = 298 # 25C -> 298K
# coordinates range & initial matrix
myrange = 120
mymat = np.zeros((myrange,myrange,myrange))

In [ ]:
# get subfolders
path = [name for name in os.listdir(".") if os.path.isdir(name)]
l = 0 # number of folders averaged
# load reference
mol_ref = Molecule('./2x16/structure.pdb')
mol_ref.center('protein') 

In [ ]:
# run for every folder
for name in path:
    path_pdb = name + '/structure.pdb'
    path_traj = name + '/traj.xtc'
    print(path_pdb)
    try:
        mol = Molecule(path_pdb)
        mol.read(path_traj) 
        l += 1
        mymat2 = generate_mat(mol,mol_ref)
        mymat += mymat2
        #np.save(name,mymat2) # save matrix for each folder's structure
    except:
        pass


In [ ]:
# normalization and filtering
mymat_norm = mymat/l
np.save('average_prob',mymat_norm)
G = - kB * T * np.log(mymat_norm) # apply free energy
mymat_norm_filt = gaussian_filter(mymat_norm, 1.5)
G2 = - kB * T * np.log(mymat_norm_filt) # apply free energy

In [ ]:
# create cube object
mi = np.asarray([-int(myrange/2),-int(myrange/2),-int(myrange/2)]) # min coords
ma = np.asarray([int(myrange/2),int(myrange/2),int(myrange/2)]) # max coords
res = np.asarray([1,1,1]) # resolution
molecule.util.writeVoxels(G,'final.cube',mi,ma,res) # average cube
molecule.util.writeVoxels(G2,'final_filtered.cube',mi,ma,res) # filtered average cube